# CHAPTER 11 동적 계획법

**동적 계획법** : 복잡한 문제를 재귀를 통해 간단한 하위 문제로 분류하여 단순화하여 해결하는 방법

어떤 문제가 *최적 부분 구조* 와 *중복되는 부분 문제* 를 갖고 있다면 동적 계획법으로 해결 가능

- 최적 부분 구조 : 답을 구하기 위해서 했던 계산을 반복해야 하는 문제의 구조

### 11.1 메모이제이션

프로그램이 동일한 계산을 반복할 때, 이전에 계산한 값을 메모리에 저장하여 동일한 계산의 반복 수행을 제거하여 프로그램의 실행 속도를 빠르게 하는 기법


In [ ]:
from functools import wraps
import time

# 시간 측정하는 함수
def benchmark(method):
    @wraps(method)
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print("%r: %2.2f ms" % (method.__name__, (te-ts) * 1000))
        # print(f"{method.__name__}: {((te-ts)*1000):.2f} ms")
        # print("{0}: {1:0.2f} ms".format(method.__name__, ((te-ts)*1000))) 
        # 위에 print 3개 다 같은 결과
        
        return result
    
    return timed

In [ ]:
# 메모이제이션을 활용한 피보나치 수열
from functools import wraps

# from benchmark import benchmark
def memo(func):
    cache = {}  # 캐시

    @wraps(func)    
    def wrap(*args):
        if args not in cache:    # 캐시에 있는 값들은 그대로 불러오고 아닌 값들만 계산
            cache[args] = func(*args)
        return cache[args]
    return wrap

def fib(n):     # 일반적인 fib(n)
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)
    
@memo           # 캐시 사용
def fib2(n):
    if n < 2:
        return 1
    else:
        return fib2(n-1) + fib2(n-2)
    
def fib3(m,n):  # m 은 list
    if m[n] == 0:
        m[n] = fib3(m,n-1) + fib3(m,n-2)
    return m[n]
    
@benchmark  # 함수 실행에 걸린 시간 측정하는 데코레이터
def test_fib(n):
    print(fib(n))

@benchmark
def test_fib2(n):
    print(fib2(n))

@benchmark
def test_fib3(n):
    m = [0] * (n+1)
    m[0], m[1] = 1, 1   # m -> [1,1,0,0,0 ... ]
    print(fib3(m,n))

if __name__ == "__main__":
    n = 35
    test_fib(n)
    test_fib2(n)
    test_fib3(n)

### 11.2 연습문제

최장 증가 부분열 : 

증가하는 부분열의 길이가 최대가 되게 하는 부분열

In [ ]:
from bisect import bisect
from itertools import combinations
from functools import wraps

def naive_longest_inc_subseq(seq):
    """ 1) 단순한 방법 """
    for length in range(len(seq), 0, -1):
        for sub in combinations(seq, length):   # combination(seq, length): n C(length) -> 모든 조합
            if list(sub) == sorted(sub):        # combination 해도 순서는 지켜지기 때문
                return len(sub)

def dp_longest_inc_subseq(seq):
    """ 2) 동적 계획법 """
    L = [1] * len(seq)
    res = []    # 캐시
    for cur, val in enumerate(seq):    # cur(index), val(value)
        for pre in range(cur):
            if seq[pre] <= val:
                L[cur] = max(L[cur], 1 + L[pre])
    return max(L)

def memo(func):
    cache = {}

    @wraps(func)
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

def memoized_longest_inc_subseq(seq):
    """ 3) 메모제이션 """
    @memo
    def L(cur):
        res = 1
        for pre in range(cur):
            if seq[pre] <= seq[cur]:
                res = max(res, 1 + L(pre))
        return res
    return max(L(i) for i in range(len(seq)))
    